In [1]:
!pip install torch
!pip install torchtext
!pip install torchdata

In [4]:
import torch
import torchtext
import torchdata
from torchtext.datasets import IMDB

train_iter = torchtext.datasets.IMDB(root = './', split ='train')
test_iter =  torchtext.datasets.IMDB(root = './', split ='test')

In [17]:
for label,text in test_iter:
    print ('label:',label,'\ntext:', text)
    break

label: neg 
text: I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn't match the background, and painfully one-dimensional characters cannot be overcome with a 'sci-fi' setting. (I'm sure there are those of you out there who think Babylon 5 is good sci-fi TV. It's not. It's clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It's really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it's rubbish as 

In [5]:
#Saving a part of training data in json object
import json

train_data = {'text':[],'label':[]}

f = open('./datasets/IMDB/aclImdb_v1/train/pos','r')
count = 0
for line in f:
    train_data['text'].append(line)
    train_data['label'].append(1)
    count += 1
    if (count>= 50):
        break
f.close()

f = open('./datasets/IMDB/aclImdb_v1/train/neg','r')
count = 0
for line in f:
    train_data['text'].append(line)
    train_data['label'].append(0)
    count += 1
    if (count>= 50):
        break
f.close()

#print ((train_data['label']))
with open('./datasets/IMDB/train_data.json', 'w') as fp:
    json.dump(train_data, fp)

In [6]:
#Saving a part of test data in json object
import json

test_data = {'text':[],'label':[]}

f = open('./datasets/IMDB/aclImdb_v1/test/pos','r')
count = 0
for line in f:
    test_data['text'].append(line)
    test_data['label'].append(1)
    count += 1
    if (count>= 50):
        break
f.close()

f = open('./datasets/IMDB/aclImdb_v1/test/neg','r')
count = 0
for line in f:
    test_data['text'].append(line)
    test_data['label'].append(0)
    count += 1
    if (count>= 50):
        break
f.close()

#print ((train_data['label']))
with open('./datasets/IMDB/test_data.json', 'w') as fp:
    json.dump(test_data, fp)

In [7]:
#Now we need to create a vocabulary from our train text
#We need two things (i) A tokenizer that will tokenize a given text (ii) A counter object that will count occurance of tokens

from torchtext.data import get_tokenizer
import collections

tokenizer = get_tokenizer("basic_english")
counter_obj = collections.Counter()

for atext in train_data['text']:
    tokens = tokenizer(atext)
    counter_obj.update(tokens)
    
from torchtext.vocab import vocab

vocabobj = vocab(counter_obj, min_freq = 1, specials=['<unk>'] )
vocabobj.set_default_index(vocabobj['<unk>'])
vocabobj

Vocab()

In [8]:
vocabobj.get_stoi()


{'shorter': 3996,
 'apparent': 3995,
 'drag': 3994,
 'lullaby': 3993,
 'victim': 3990,
 'episode': 3989,
 'fill': 3988,
 'cater': 3986,
 'japp': 3985,
 'students': 3982,
 '50s': 3981,
 'march': 3980,
 'jarrow': 3979,
 'painted': 3978,
 '1930s': 3975,
 '1950s': 3974,
 'minded': 3973,
 'lmn': 3972,
 'supporting': 3971,
 'pfieffer': 3969,
 'deedee': 3968,
 'ratings': 3962,
 'peroxide': 3961,
 'wall': 3959,
 'brick': 3958,
 'hours': 3956,
 'thriller': 3954,
 'senseless': 3952,
 'hike': 3950,
 'raft': 3940,
 'spoiled': 3935,
 'grousing': 3934,
 'whining': 3933,
 'sean--and': 3930,
 'confesses': 3928,
 'fault': 3927,
 'whines': 3926,
 'cheating': 3925,
 'confrontation': 3924,
 'aggressive': 3923,
 'heavy': 3922,
 'upset': 3917,
 'phones': 3915,
 'divorced': 3914,
 'older': 3913,
 'mancuso': 3912,
 'manager': 3910,
 'hotel': 3909,
 'out-standing': 3907,
 'dots': 3905,
 'connect': 3904,
 'covered': 3902,
 'whine': 3899,
 'zack': 3895,
 'week': 3894,
 'slap': 3893,
 'insufferable': 3892,
 'cell

In [9]:
vocabobj['randomtext']

0

In [10]:
len(train_data['label'])

100

In [42]:
def trp(l, n):
    return l[:n] + [0]*(n-len(l))

inp_len = 100
text_pipeline = lambda x: trp(vocabobj(tokenizer(x)),inp_len)
label_pipeline = lambda x: x

for aline in train_data['text']:
    print (len(text_pipeline(aline)))
    break


100


In [90]:
# We need dataloader to fetch a batch of data for training 

from torch.utils.data import Dataset, DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from torch.nn import functional as F

class CustomTextDataset(Dataset):
    def __init__(self, text, labels):
        self.labels = labels
        self.text = text
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        label = self.labels[idx]
        text = self.text[idx]
        sample = {"Text": text, "labels": label}
        return sample

TrainingDataObj = CustomTextDataset(train_data['text'], train_data['label'])
TestDataObj= CustomTextDataset(test_data['text'], test_data['label'])

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    
    for asample in batch:
        label_list.append(label_pipeline(asample['labels']))
        #print (label_list)
        processed_text = torch.tensor(text_pipeline(asample['Text']) , dtype=torch.int64)  
        #print (processed_text.size())
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
        
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    #text_list = torch.stack(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

Traindataloader = DataLoader(TrainingDataObj, batch_size=5, shuffle=True, collate_fn= collate_batch)

for idx, (label, text, offsets) in enumerate(Traindataloader):
    print(text.size())
    print(label)
    print(offsets.int())
    break
    


torch.Size([500])
tensor([1, 1, 1, 1, 0])
tensor([  0, 100, 200, 300, 400], dtype=torch.int32)


In [54]:
TrainingDataObj[0]

{'Text': 'Zentropa has much in common with The Third Man, another noir-like film set among the rubble of postwar Europe. Like TTM, there is much inventive camera work. There is an innocent American who gets emotionally involved with a woman he doesn\'t really understand, and whose naivety is all the more striking in contrast with the natives.<br /><br />But I\'d have to say that The Third Man has a more well-crafted storyline. Zentropa is a bit disjointed in this respect. Perhaps this is intentional: it is presented as a dream/nightmare, and making it too coherent would spoil the effect. <br /><br />This movie is unrelentingly grim--"noir" in more than one sense; one never sees the sun shine. Grim, but intriguing, and frightening.\n',
 'labels': 1}

In [96]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [97]:
num_class = len(set(TrainingDataObj.labels))
vocab_size = len(vocabobj)
emsize = 64
input_size = inp_len
hidden_size=20
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [98]:
len(vocabobj)

3997

In [99]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count


In [100]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None


train_dataloader = DataLoader(TrainingDataObj, batch_size=2, shuffle=True, collate_fn= collate_batch)
test_dataloader = DataLoader(TestDataObj, batch_size=2, shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(test_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'test accuracy {:8.3f} '.format(epoch, time.time() - epoch_start_time, accu_val))
    print('-' * 59)


-----------------------------------------------------------
| end of epoch   1 | time:  0.08s | test accuracy    0.560 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   2 | time:  0.06s | test accuracy    0.610 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   3 | time:  0.06s | test accuracy    0.580 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   4 | time:  0.06s | test accuracy    0.520 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   5 | time:  0.07s | test accuracy    0.520 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   6 | time:  0.06s | test